In [1]:
import pandas as pd
import numpy as np
import pprint

url='https://drive.google.com/file/d/1PCJ7ltluquoXKi6MYTPMfwZQNI_-MIFP/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
data_frame = pd.read_csv(url)

In [2]:
answer_dict =  {"Q1" : None,
                "Q2" : None,
                "Q3" : None,
                "Q4" : None,
                "Q5" : None,
                "Q6" : None,
                "Q7" : None}

In [3]:
# ######### Q1

vw_frame = data_frame[data_frame["Make"] == 'Volkswagen']
mean_co2_vw = vw_frame['CO2 Emission Grams/Mile'].mean()

answer_dict["Q1"] =  mean_co2_vw

In [4]:
# ######### Q2

# Separate from the duplicate car models.
unique_models = data_frame.drop_duplicates(subset = ["Model"]) 

# Count duplicates in brand to know how many unique models the brand have.
brand_count = unique_models.pivot_table(columns = ['Make'], aggfunc='size')

# Order the brands by number of models
top5_models = brand_count.sort_values(ascending = False)
brand_count = top5_models.head(5)

# Get the indexes' names
indexes = []
for row in brand_count.head().index:
    indexes.append(row)
    
# Get the values
values = brand_count.values.tolist()

top5_list = [[indexes[i], values[i]] for i in range(len(indexes))]
#pprint.pprint (top5_list)

answer_dict["Q2"] = top5_list 

In [5]:
########## Q3

fuel_types = list(data_frame['Fuel Type'].unique())
fuel_types.sort()

answer_dict["Q3"] =  fuel_types

# Doing this excersice I realized there were some Fuel Types strings that have more words describing other types of fuels,
# so I decided to code other way to separate those strings and have a more accurate answer, describing separately ALL types
# of fuels.
#
# Here my other code:
#
# fuel_types = (data_frame['Fuel Type'].unique())
#
# other_fuels1 = [] # Create list to save the joined fuels like "Gasoline or E85" or "Premium and electricity"
# other_fuels2 = [] # List for singular fuels
#
# for fuels in fuel_types:
#     if "or" in fuels or "and" in fuels:
#         o = fuels.find('or') 
#         y = fuels.find('and')
#         if o > 0:
#             other_fuels1.append(fuels[:o-1])
#             other_fuels1.append(fuels[o+3:])
#         else:
#             other_fuels1.append(fuels[:y-1])
#             other_fuels1.append(fuels[y+4:])
#     else:
#         other_fuels2.append(fuels)
#
# capitalized_fuels = []
#
# for item in other_fuels1:
#     capitalized_fuels.append(item.capitalize())
#
# all_fuels = list(set(capitalized_fuels + other_fuels2))
# all_fuels.sort()


In [6]:
########## Q4

# Import module to calculate z score
import scipy.stats as stats

toyota_frame = data_frame[data_frame["Make"] == 'Toyota']
z_score = stats.zscore(toyota_frame['Fuel Barrels/Year'])

# Add a new colum with z scores to toyota_frame
toyota_frame.insert(loc = 10, column = 'z score', value = z_score)

 # Add colum for absolute values of z score to toyota_frame
toyota_frame.insert(loc = 11, column = '|z score|', value = abs(z_score))
top9 = toyota_frame.nlargest(9,'|z score|')
top9_sort = top9.sort_values(by = ['|z score|', 'Year'], ascending = False)

# Create columns lists to form 9X3 list
models_ls = top9_sort['Model'].values.tolist()
years_ls = top9_sort['Year'].values.tolist()
zscore_ls = top9_sort['z score'].values.tolist()

# Join lists
top9_list = [[models_ls[i], years_ls[i],zscore_ls[i]] for i in range(9)]


answer_dict["Q4"] =  top9_list

In [7]:
########## Q5

# Filter data
golf_frame = data_frame[(data_frame["Model"] == 'Golf') & (data_frame["Transmission"] == 'Manual 5-spd') & (data_frame["Fuel Type"] == 'Regular')]

# Calculate the difference ann add a column in golf_frame
golf_frame.insert(loc = 13, column = 'Combined MPG difference', value = golf_frame['Combined MPG'].diff())

# Avoid "NaN"
golf_frame.fillna(0, inplace=True)

# Create columns lists to form 19X3 list
years_ls = golf_frame['Year'].values.tolist()
mpg_ls = golf_frame['Combined MPG'].values.tolist()
diff_ls = golf_frame['Combined MPG difference'].values.tolist()

# Join lists
mpg_golfs = [[years_ls[i], mpg_ls[i],diff_ls[i]] for i in range(19)]

answer_dict["Q5"] =  mpg_golfs

C:\Users\maryf\Anaconda3\lib\site-packages\pandas\core\frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [8]:
########## Q6

brand_list = ['Toyota','Ford','Volkswagen','Nissan','Honda']
top5_co2_emissions = []

for brand in brand_list:
    brand_frame = data_frame[data_frame["Make"] == brand]
    top5 = brand_frame.nsmallest(5,'CO2 Emission Grams/Mile')
    brand_co2 = list(top5['CO2 Emission Grams/Mile'])
    brand_co2.insert(0,brand)
    top5_co2_emissions.append(brand_co2)
    
answer_dict["Q6"] =  top5_co2_emissions

In [9]:
########## Q7

import statistics

years = [(1984, 1988), (1989, 1993), (1994, 1998), (1999, 2003), (2004, 2008), (2009, 2013), (2014, 2018)]
median_ls=[]

for i in range(len(years)):
    tup = years[i]
    group = data_frame.loc[(data_frame['Year'] >= tup[0]) & (data_frame['Year'] <= tup[1])]
    median_ls.append(float(statistics.median(group['Combined MPG'])))


join_list = [[years[i], median_ls[i]] for i in range(7)]

answer_dict["Q7"] =  join_list

In [10]:
#pprint.pprint (answer_dict)